<a href="https://colab.research.google.com/github/fallensnake/Calsolstuff/blob/main/ResearchAnalysis_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import json
from openai import OpenAI
from google.colab import userdata
import re

In [14]:

# --- CONFIGURATION ---
# You need a PERPLEXITY_API_KEY in your Colab Secrets
# Get one here: https://www.perplexity.ai/pplx-api
PPLX_API_KEY = "[REDACTED]"

client = OpenAI(
    api_key=PPLX_API_KEY,
    base_url="https://api.perplexity.ai"
)

def get_ai_analysis(market_question, current_price, end_date):
    """
    Asks Perplexity to research a specific market and estimate probability.
    """

    # 1. Craft the "Analyst Persona" Prompt
    system_prompt = """
    You are a ruthless prediction market analyst. Your goal is to estimate the
    TRUE probability of an event occurring based on the latest news.

    You will be given a market question and its current trading price (implied probability).
    Your output must be a strict JSON object with NO markdown formatting.

    JSON Format:
    {
        "my_estimated_probability": 0.0 to 1.0,
        "reasoning": "A short, dense summary of why (max 2 sentences).",
        "verdict": "BUY_YES", "BUY_NO", or "PASS",
        "confidence": "HIGH" or "LOW"
    }

    Rules for Verdict:
    - BUY_YES if your prob > (current_price + 0.10)
    - BUY_NO if your prob < (current_price - 0.10)
    - PASS if the edge is too small or news is unclear.
    """

    user_prompt = f"""
    Event: "{market_question}"
    Current Market Price for YES: {current_price} cents (Implied Prob: {current_price/100})
    Expiration Date: {end_date}

    Search the web for the latest news on this topic.
    Compare the market's implied probability vs. reality.
    """

    try:
        # 2. Call Perplexity (sonar-reasoning-pro is best for deep research)
        # 'sonar' or 'sonar-pro' are faster/cheaper alternatives
        response = client.chat.completions.create(
            model="sonar-reasoning-pro",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1, # Keep it factual
        )

        # 3. Parse the JSON
        content = response.choices[0].message.content

        # 1. Use Regex to find the first '{' and the last '}'
        # This ignores any "Here is your JSON:" chatter from the AI
        json_match = re.search(r'\{.*\}', content, re.DOTALL)

        if json_match:
            clean_content = json_match.group(0)
            analysis = json.loads(clean_content)
            return analysis
        else:
            print(f"❌ No JSON found in response for: {market_question}")
            print(f"Full response was: {content}")
            return None

    except json.JSONDecodeError as e:
        print(f"⚠️ JSON failed to parse for '{market_question}': {e}")
        # Log the content so you can see what broke it
        print(f"Offending content: {content}")
        return None
    except Exception as e:
        print(f"⚠️ Research failed: {e}")
        return None

def research_batch(markets_list):
    """
    Takes a list of raw markets (from the Scout) and filters for GOLD.
    """
    print(f"\n🔬 Starting Deep Research on {len(markets_list)} markets...")
    print("-" * 60)

    opportunities = []

    for market in markets_list:
        question = market['question']
        price = market['yes_price']

        # Skip if price is too extreme (e.g., 99c or 1c) to avoid liquidity traps
        if price < 5 or price > 95:
            continue

        print(f"Searching: {question[:50]}...")

        analysis = get_ai_analysis(question, price, market['end_date'])

        if analysis and analysis['verdict'] != 'PASS':
            # We found an edge!
            opportunity = {
                "market": market,
                "analysis": analysis,
                "edge": abs(analysis['my_estimated_probability'] - (price/100))
            }
            opportunities.append(opportunity)

            # Real-time feedback
            print(f"  >>> FOUND EDGE: {analysis['verdict']} (Conf: {analysis['confidence']})")
            print(f"      Reason: {analysis['reasoning']}")

    return opportunities


In [15]:
if __name__ == "__main__":
    # Fake data to test the logic without running the full Scout
    test_markets = [
        {"question": "Will Bitcoin be above $200k by next year?", "yes_price": 15, "end_date": "2026-12-31"},
        {"question": "Will it snow in New York tomorrow?", "yes_price": 10, "end_date": "2026-01-08"}
    ]

    results = research_batch(test_markets)

    print("\n" + "="*50)
    print("🚀 FINAL ADVISOR REPORT")
    print("="*50)
    for op in results:
        m = op['market']
        a = op['analysis']
        print(f"Event: {m['question']}")
        print(f"  Strategy: {a['verdict']} @ {m['yes_price']}¢")
        print(f"  AI Edge: {op['edge']*100:.1f}%")
        print(f"  Note: {a['reasoning']}")
        print("-" * 30)


🔬 Starting Deep Research on 2 markets...
------------------------------------------------------------
Searching: Will Bitcoin be above $200k by next year?...
Searching: Will it snow in New York tomorrow?...
  >>> FOUND EDGE: BUY_YES (Conf: LOW)
      Reason: FOX Weather explicitly shows rain AND snowfall expected for Friday, Jan 9, 2026[5], and NOAA forecasts above-normal precipitation for Jan 3-9 with near-seasonal temperatures[1]. However, Weather25 shows borderline temps (3°C/-2°C) on Jan 9[2], making rain vs snow uncertain.

🚀 FINAL ADVISOR REPORT
Event: Will it snow in New York tomorrow?
  Strategy: BUY_YES @ 10¢
  AI Edge: 35.0%
  Note: FOX Weather explicitly shows rain AND snowfall expected for Friday, Jan 9, 2026[5], and NOAA forecasts above-normal precipitation for Jan 3-9 with near-seasonal temperatures[1]. However, Weather25 shows borderline temps (3°C/-2°C) on Jan 9[2], making rain vs snow uncertain.
------------------------------
